In [91]:
import pandas as pd
import math
import time

## translation libraries
import translators as ts
from googletrans import Translator as gt
from deep_translator import GoogleTranslator
from translate import Translator as tlr

## sentiment analysis libraries
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# nltk.download('all')

In [6]:
import pandas as pd

language_files = [
    "TMRC14_APAC_1", "TMRC14_APAC_2", "TMRC14_APAC_3", 
    "twitterrei_china_052020", "twitterrei_china_082019_1",
    "twitterrei_china_082019_2", "twitterrei_china_082019_3", 
    "twitterrei_thailand_092020"
]
root = '/Users/rlzhang1310/Coding/buntain/language_data/'
end = '.csv'

top_languages = pd.DataFrame()

for file in language_files:
    src = root + file + end
    df = pd.read_csv(src)
    if "tweet_language" in df.columns:
        df["language"] = df["tweet_language"]
        df.drop(["tweet_language"], axis=1, inplace=True)
    top_languages = pd.concat([top_languages, df], axis=0)

top_languages = top_languages.groupby('language').sum().reset_index()
top_languages = top_languages.sort_values(by='count', ascending=False)
total_count = top_languages["count"].sum()

top_languages_lst = top_languages.head(7)['language'].tolist()
top_languages_lst


['en', 'in', 'ur', 'zh', 'und', 'ar', 'pt']

In [7]:
count = top_languages[top_languages['language'].isin(top_languages_lst)]["count"].sum()
print(count / total_count)

0.914366900848165


In [8]:
## spacy sentiment analysis
en_nlp = spacy.load('en_core_web_sm')
en_nlp.add_pipe('spacytextblob')

def get_spacy_sentiment(text, en_nlp):
    doc = en_nlp(text)
    return doc._.blob.polarity   

/Users/rlzhang1310/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [99]:
def ts_tester(text, to):
    retries = 5
    delay=1
    for attempt in range(retries):
        try:
            to_in = ts.translate_text(text, from_language='en', to_language=to)
            break
        except Exception as e:
            print(f"Error during translation to {to}: {e}. Retrying... ({attempt + 1}/{retries})")
            time.sleep(delay)
    else:
        print(f"Failed to translate text '{text}' to {to} after {retries} attempts.")
        return None

    # Try translating back to English
    for attempt in range(retries):
        try:
            back_en = ts.translate_text(to_in, from_language=to, to_language='en')
            break
        except Exception as e:
            print(f"Error during back translation to English: {e}. Retrying... ({attempt + 1}/{retries})")
            time.sleep(delay)
    else:
        print(f"Failed to back translate text '{to_in}' to English after {retries} attempts.")
        return None

    return get_spacy_sentiment(back_en, en_nlp)
    # to_in = ts.translate_text(text, from_language='en',to_language=to)
    # back_en = ts.translate_text(to_in,from_language=to, to_language='en')
    # return get_spacy_sentiment(back_en, en_nlp)

In [92]:
def googletrans_tester(text, to):
    googletrans_translator = gt()
        # Try translating from English to the target language
    retries =5
    delay= 10
    for attempt in range(retries):
        try:
            to_in_trans = googletrans_translator.translate(text, src='en', dest=to)
            to_in = to_in_trans.text
            break
        except Exception as e:
            print(f"Error during translation to {to}: {e}. Retrying... ({attempt + 1}/{retries})")
            time.sleep(delay)
    else:
        print(f"Failed to translate text '{text}' to {to} after {retries} attempts.")
        return None

    # Try translating back to English
    for attempt in range(retries):
        try:
            back_en = googletrans_translator.translate(to_in, src=to, dest='en')
            break
        except Exception as e:
            print(f"Error during back translation to English: {e}. Retrying... ({attempt + 1}/{retries})")
            time.sleep(delay)
    else:
        print(f"Failed to back translate text '{to_in}' to English after {retries} attempts.")
        return None

    return get_spacy_sentiment(back_en.text, en_nlp)

In [12]:
def deep_tester(text, to):
    to_in = GoogleTranslator(source='en', target=to).translate(text)
    back_en = GoogleTranslator(source=to, target='en').translate(to_in)
    return get_spacy_sentiment(back_en, en_nlp)

In [66]:
email = "rlzhang49@gmail.com"
def translate_tester(text, to):
    print(text)
    to_in = tlr(from_lang='en', to_lang=to, email=email).translate(text)
    print(to_in)
    back_en = tlr(from_lang=to,to_lang="en", email=email).translate(to_in)
    print(back_en)
    return get_spacy_sentiment(back_en, en_nlp)

In [14]:
## https://www.kaggle.com/datasets/saurabhshahane/twitter-sentiment-dataset
twitter_data = pd.read_csv("/Users/rlzhang1310/Coding/buntain/twitter_dataset/Twitter_Data.csv")
twitter_data["text"] = twitter_data["clean_text"]
twitter_data["score"] = twitter_data["category"]
twitter_data = twitter_data.drop(["clean_text", "category"], axis=1)
twitter_data.dropna(inplace=True)

In [15]:
twitter_data = twitter_data.sample(n=1000)
twitter_data["spacy_sentiment"] = twitter_data["text"].apply(get_spacy_sentiment, en_nlp=en_nlp)
twitter_data

,text,score,spacy_sentiment
76734,₹72000 election promise modi promised lakh dur...,0.0,0.000
105563,what does amazing interview modi with arnab go...,1.0,0.600
46243,congratulations the government why dont like t...,0.0,0.000
147520,hit like you think modi will our next also,0.0,0.000
79709,this needs reach many people possible narendra...,1.0,0.250
...,...,...,...
83397,bhakts can say tomorrow ask them who give birt...,0.0,0.000
121606,ensureed this north korea than what happens wi...,-1.0,-0.800
154144,deepender losing because these factors jat rio...,0.0,0.000
159522,are very afraid modi ghe man his words bolta h...,-1.0,-0.780


In [93]:
head = twitter_data.head(5)
head["testing"] = head.apply(lambda row: googletrans_tester(row['text'], to='id'), axis=1)
head


/var/folders/r4/sgr1zrxx0038xwsxj6n3k8900000gn/T/ipykernel_20166/1266798558.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  head["testing"] = head.apply(lambda row: googletrans_tester(row['text'], to='id'), axis=1)


,text,score,spacy_sentiment,id_ts,id_gs,id_dp,id_tr,ur_ts,testing
76734,₹72000 election promise modi promised lakh dur...,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0
105563,what does amazing interview modi with arnab go...,1.0,0.60,0.75,0.6,0.6,0.6,0.60,0.6
46243,congratulations the government why dont like t...,0.0,0.00,0.00,0.0,0.0,0.0,-0.30,0.0
147520,hit like you think modi will our next also,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0
79709,this needs reach many people possible narendra...,1.0,0.25,0.25,0.5,0.5,0.5,0.25,0.5


In [30]:
def acc_dict():
    d = dict()
    d["rmse"] = 0
    d["mae"] = 0
    d["pos_correct"] = 0
    d["neg_correct"] = 0
    d["neu_correct"] = 0
    d["pos_total"] = 0
    d["neg_total"] = 0
    d["neu_total"] = 0
    return d
id_ts = acc_dict()
id_gs = acc_dict()
id_dp = acc_dict()
id_tr = acc_dict()

ur_ts = acc_dict()
ur_gs = acc_dict()
ur_dp = acc_dict()
ur_tr = acc_dict()

zh_ts = acc_dict()
zh_gs = acc_dict()
zh_dp = acc_dict()
zh_tr = acc_dict()

ar_ts = acc_dict()
ar_gs = acc_dict()
ar_dp = acc_dict()
ar_tr = acc_dict()

In [31]:
def calculation_helper(actual, output, acc):
    diff = abs(output - actual)
    acc["rmse"] += diff**2
    acc["mae"] += diff
    if actual == 0:
        acc["neu_total"] += 1
        if output == 0:
            acc["neu_correct"] += 1
    elif actual > 0:
        acc["pos_total"] += 1
        if output > 0:
            acc["pos_correct"] += 1
    else:
        acc["neg_total"] += 1
        if output < 0:
            acc["neg_correct"] += 1

In [19]:
twitter_data["id_ts"] = twitter_data.apply(lambda row: ts_tester(row['text'], to='id'), axis=1)


In [27]:
twitter_data["id_gs"] = twitter_data.apply(lambda row: googletrans_tester(row["text"], to='id'), axis=1)


In [28]:
twitter_data["id_dp"] = twitter_data.apply(lambda row: deep_tester(row["text"], to='id'), axis=1)


In [29]:
twitter_data["id_tr"] = twitter_data.apply(lambda row: translate_tester(row["text"], to='id'), axis=1)

Indonesian Score

In [33]:
twitter_data.apply(lambda row: calculation_helper(row["spacy_sentiment"], row["id_ts"], id_ts), axis=1)
twitter_data.apply(lambda row: calculation_helper(row["spacy_sentiment"], row["id_gs"], id_gs), axis=1)
twitter_data.apply(lambda row: calculation_helper(row["spacy_sentiment"], row["id_dp"], id_dp), axis=1)
twitter_data.apply(lambda row: calculation_helper(row["spacy_sentiment"], row["id_tr"], id_tr), axis=1)


76734     None
105563    None
46243     None
147520    None
79709     None
          ... 
83397     None
121606    None
154144    None
159522    None
144468    None
Length: 1000, dtype: object

In [81]:
dropped_invalid_tr = twitter_data[twitter_data["id_tr"] != 0.34]
print(len(dropped_invalid_tr))
id_test_ts = acc_dict()
id_test_gs = acc_dict()
id_test_dp = acc_dict()
id_test_tr = acc_dict()
dropped_invalid_tr.apply(lambda row: calculation_helper(row["spacy_sentiment"], row["id_ts"], id_test_ts), axis=1)
dropped_invalid_tr.apply(lambda row: calculation_helper(row["spacy_sentiment"], row["id_gs"], id_test_gs), axis=1)
dropped_invalid_tr.apply(lambda row: calculation_helper(row["spacy_sentiment"], row["id_dp"], id_test_dp), axis=1)
dropped_invalid_tr.apply(lambda row: calculation_helper(row["spacy_sentiment"], row["id_tr"], id_test_tr), axis=1)

def print_res(d):
    print(f'rmse: {math.sqrt(d["rmse"])}, mae: {d["mae"]}')
    print(f'positive accuracy: {round(d["pos_correct"] / d["pos_total"], 4)}, '
          f'negative accuracy: {round(d["neg_correct"] / d["neg_total"], 4)}, '
          f'neutral accuracy: {round(d["neu_correct"] / d["neu_total"], 4)}, '
          f'total accuracy: {round((d["pos_correct"] + d["neg_correct"] + d["neu_correct"]) / (d["pos_total"] + d["neg_total"] + d["neu_total"]), 4)}')
    
print("\nts results")
print_res(id_test_ts)
print("\ngs results")
print_res(id_test_gs)
print("\ndp results")
print_res(id_test_dp)
print("\ntr results")
print_res(id_test_tr)

68

ts results
rmse: 1.7414958528881879, mae: 6.792227032227032
positive accuracy: 0.9706, negative accuracy: 0.75, neutral accuracy: 0.8182, total accuracy: 0.8824

gs results
rmse: 0.9057156647430747, mae: 4.330768398268397
positive accuracy: 0.9706, negative accuracy: 0.75, neutral accuracy: 0.8636, total accuracy: 0.8971

dp results
rmse: 1.8452191484652671, mae: 6.826931216931217
positive accuracy: 0.8824, negative accuracy: 0.75, neutral accuracy: 0.8636, total accuracy: 0.8529

tr results
rmse: 1.1275553790427277, mae: 4.128578643578644
positive accuracy: 0.8824, negative accuracy: 0.9167, neutral accuracy: 1.0, total accuracy: 0.9265


In [40]:
# def print_res(d):
#     print(f'rmse: {math.sqrt(d["rmse"])},  mae: {d["mae"]}')
#     print(f'positive accuracy: {round(d["pos_correct"] / d["pos_total"], 4)}, negative accuracy: {round(d["neg_correct"] / d["neg_total"], 4)}, 
#           neutral accuracy: {round(d["neu_correct"] / d["neu_total"], 4)}, 
#           total accuracy: {round(d["pos_correct"] + d["neg_correct"] + d["neu_correct"] / d["pos_total"] + d["neg_total"] + d["neu_total"])}')

def print_res(d):
    print(f'rmse: {math.sqrt(d["rmse"])}, mae: {d["mae"]}')
    print(f'positive accuracy: {round(d["pos_correct"] / d["pos_total"], 4)}, '
          f'negative accuracy: {round(d["neg_correct"] / d["neg_total"], 4)}, '
          f'neutral accuracy: {round(d["neu_correct"] / d["neu_total"], 4)}, '
          f'total accuracy: {round((d["pos_correct"] + d["neg_correct"] + d["neu_correct"]) / (d["pos_total"] + d["neg_total"] + d["neu_total"]), 4)}')
    
print("\nts results")
print_res(id_ts)
print("\ngs results")
print_res(id_gs)
print("\ndp results")
print_res(id_dp)
print("\ntr results")
print_res(id_tr)


ts results
rmse: 5.210072332618934, mae: 79.32066892906478
positive accuracy: 0.8764, negative accuracy: 0.8, neutral accuracy: 0.8484, total accuracy: 0.849

gs results
rmse: 5.906385483899573, mae: 100.24026441860987
positive accuracy: 0.818, negative accuracy: 0.698, neutral accuracy: 0.829, total accuracy: 0.792

dp results
rmse: 5.624881635834863, mae: 90.8202041992399
positive accuracy: 0.8674, negative accuracy: 0.8082, neutral accuracy: 0.7742, total accuracy: 0.824

tr results
rmse: 11.803534465010184, mae: 310.17673451548313
positive accuracy: 0.991, negative accuracy: 0.0449, neutral accuracy: 0.071, total accuracy: 0.474


In [87]:
twitter_data["ur_ts"] = twitter_data.apply(lambda row: ts_tester(row["text"], to='ur'), axis=1)


In [94]:
twitter_data["ur_gs"] = twitter_data.apply(lambda row: googletrans_tester(row["text"], to='ur'), axis=1)


Error during translation to ur: the JSON object must be str, bytes or bytearray, not NoneType. Retrying... (1/5)
Error during translation to ur: the JSON object must be str, bytes or bytearray, not NoneType. Retrying... (1/5)
Error during back translation to English: the JSON object must be str, bytes or bytearray, not NoneType. Retrying... (1/5)


In [95]:
twitter_data["ur_dp"] = twitter_data.apply(lambda row: deep_tester(row["text"], to='ur'), axis=1)


In [ ]:
# twitter_data["ur_tr"] = twitter_data.apply(lambda row: translate_tester(row["text"], to='ur'), axis=1)

In [96]:
twitter_data.apply(lambda row: calculation_helper(row["spacy_sentiment"], row["ur_ts"], ur_ts), axis=1)
twitter_data.apply(lambda row: calculation_helper(row["spacy_sentiment"], row["ur_gs"], ur_gs), axis=1)
twitter_data.apply(lambda row: calculation_helper(row["spacy_sentiment"], row["ur_dp"], ur_dp), axis=1)
# twitter_data.apply(lambda row: calculation_helper(row["spacy_sentiment"], row["ur_tr"], ur_tr), axis=1)

76734     None
105563    None
46243     None
147520    None
79709     None
          ... 
83397     None
121606    None
154144    None
159522    None
144468    None
Length: 1000, dtype: object

In [97]:
print("\nts results")
print_res(ur_ts)
print("\ngs results")
print_res(ur_gs)
print("\ndp results")
print_res(ur_dp)
# print("\ntr results")
# print_res(ur_tr)


ts results
rmse: 6.18293813360281, mae: 106.68266653254145
positive accuracy: 0.8067, negative accuracy: 0.6857, neutral accuracy: 0.8032, total accuracy: 0.776

gs results
rmse: 7.012306690781921, mae: 126.25301932789432
positive accuracy: 0.7596, negative accuracy: 0.6245, neutral accuracy: 0.7774, total accuracy: 0.732

dp results
rmse: 5.820969520104741, mae: 97.29823829873833
positive accuracy: 0.8449, negative accuracy: 0.8041, neutral accuracy: 0.7742, total accuracy: 0.813


In [100]:
twitter_data["zh_ts"] = twitter_data.apply(lambda row: ts_tester(row["text"], to='zh'), axis=1)


Error during back translation to English: 0. Retrying... (1/5)
Error during back translation to English: 0. Retrying... (1/5)
Error during translation to zh: 0. Retrying... (1/5)
Error during translation to zh: 0. Retrying... (1/5)
Error during translation to zh: 0. Retrying... (1/5)
Error during back translation to English: 0. Retrying... (1/5)
Error during back translation to English: 0. Retrying... (1/5)
Error during translation to zh: 0. Retrying... (1/5)
Error during back translation to English: 0. Retrying... (1/5)
Error during translation to zh: 0. Retrying... (1/5)
Error during translation to zh: 0. Retrying... (1/5)
Error during back translation to English: 0. Retrying... (1/5)
Error during translation to zh: 0. Retrying... (1/5)
Error during back translation to English: 0. Retrying... (1/5)
Error during back translation to English: 0. Retrying... (1/5)
Error during translation to zh: 0. Retrying... (1/5)
Error during back translation to English: 0. Retrying... (1/5)
Error dur

In [103]:
twitter_data["zh_gs"] = twitter_data.apply(lambda row: googletrans_tester(row["text"], to='zh-CN'), axis=1)


Error during back translation to English: the JSON object must be str, bytes or bytearray, not NoneType. Retrying... (1/5)
Error during translation to zh-CN: the JSON object must be str, bytes or bytearray, not NoneType. Retrying... (1/5)
Error during back translation to English: the JSON object must be str, bytes or bytearray, not NoneType. Retrying... (1/5)
Error during translation to zh-CN: the JSON object must be str, bytes or bytearray, not NoneType. Retrying... (1/5)
Error during translation to zh-CN: the JSON object must be str, bytes or bytearray, not NoneType. Retrying... (1/5)
Error during translation to zh-CN: the JSON object must be str, bytes or bytearray, not NoneType. Retrying... (1/5)


In [105]:
twitter_data["zh_dp"] = twitter_data.apply(lambda row: deep_tester(row["text"], to='zh-CN'), axis=1)


In [ ]:
# twitter_data["zh_tr"] = twitter_data.apply(lambda row: translate_tester(row["text"], to='zh'), axis=1)

In [106]:
twitter_data.apply(lambda row: calculation_helper(row["spacy_sentiment"], row["zh_ts"], zh_ts), axis=1)
twitter_data.apply(lambda row: calculation_helper(row["spacy_sentiment"], row["zh_gs"], zh_gs), axis=1)
twitter_data.apply(lambda row: calculation_helper(row["spacy_sentiment"], row["zh_dp"], zh_dp), axis=1)

76734     None
105563    None
46243     None
147520    None
79709     None
          ... 
83397     None
121606    None
154144    None
159522    None
144468    None
Length: 1000, dtype: object

In [107]:
print("\nts results")
print_res(zh_ts)
print("\ngs results")
print_res(zh_gs)
print("\ndp results")
print_res(zh_dp)


ts results
rmse: 6.051968817849527, mae: 95.95956692084364
positive accuracy: 0.8562, negative accuracy: 0.7959, neutral accuracy: 0.7968, total accuracy: 0.823

gs results
rmse: 7.431768125160275, mae: 137.77335536916792
positive accuracy: 0.6899, negative accuracy: 0.6041, neutral accuracy: 0.7871, total accuracy: 0.699

dp results
rmse: 6.36531513004346, mae: 105.05997081699162
positive accuracy: 0.8337, negative accuracy: 0.7837, neutral accuracy: 0.7968, total accuracy: 0.81


In [108]:
twitter_data["ar_ts"] = twitter_data.apply(lambda row: ts_tester(row["text"], to='ar'), axis=1)


In [109]:
twitter_data["ar_gs"] = twitter_data.apply(lambda row: googletrans_tester(row["text"], to='ar'), axis=1)


Error during translation to ar: the JSON object must be str, bytes or bytearray, not NoneType. Retrying... (1/5)
Error during translation to ar: the JSON object must be str, bytes or bytearray, not NoneType. Retrying... (2/5)
Error during back translation to English: the JSON object must be str, bytes or bytearray, not NoneType. Retrying... (1/5)
Error during translation to ar: the JSON object must be str, bytes or bytearray, not NoneType. Retrying... (1/5)
Error during translation to ar: the JSON object must be str, bytes or bytearray, not NoneType. Retrying... (1/5)


In [110]:
twitter_data["ar_dp"] = twitter_data.apply(lambda row: deep_tester(row["text"], to='ar'), axis=1)


In [ ]:
# twitter_data["ar_tr"] = twitter_data.apply(lambda row: translate_tester(row["text"], to='ar'), axis=1)

In [111]:
twitter_data.apply(lambda row: calculation_helper(row["spacy_sentiment"], row["ar_ts"], ar_ts), axis=1)
twitter_data.apply(lambda row: calculation_helper(row["spacy_sentiment"], row["ar_gs"], ar_gs), axis=1)
twitter_data.apply(lambda row: calculation_helper(row["spacy_sentiment"], row["ar_dp"], ar_dp), axis=1)

76734     None
105563    None
46243     None
147520    None
79709     None
          ... 
83397     None
121606    None
154144    None
159522    None
144468    None
Length: 1000, dtype: object

In [112]:
print("\nts results")
print_res(ar_ts)
print("\ngs results")
print_res(ar_gs)
print("\ndp results")
print_res(ar_dp)


ts results
rmse: 5.4324834591105935, mae: 84.35008922327677
positive accuracy: 0.8607, negative accuracy: 0.8, neutral accuracy: 0.8065, total accuracy: 0.829

gs results
rmse: 6.426772199092459, mae: 113.1202286186036
positive accuracy: 0.7573, negative accuracy: 0.6082, neutral accuracy: 0.8419, total accuracy: 0.747

dp results
rmse: 5.280968149721594, mae: 78.68333919321417
positive accuracy: 0.8584, negative accuracy: 0.8245, neutral accuracy: 0.829, total accuracy: 0.841
